In [1]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# torch.cuda.is_available()
# cuda是否可用；

# torch.cuda.device_count()
# 返回gpu数量；

# torch.cuda.get_device_name(0)
# 返回gpu名字，设备索引默认从0开始；

# torch.cuda.current_device()
# 返回当前设备索引；

In [3]:
torch.cuda.current_device()

0

In [4]:
use_gpu = torch.cuda.is_available()
use_gpu

True

In [5]:
mini_batch=64
transfrom=transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize((0.1307,),(0.3081,))
                             ])

In [6]:
train_data=MNIST('./datas/mnist',
                 train=True,
                 download=False,
                 transform=transfrom)
test_data=MNIST('./datas/mnist',
                train=False,
                download=False,
                transform=transfrom)

In [7]:
train_loader=DataLoader(dataset=train_data,
                        shuffle=True,
                        batch_size=mini_batch)
test_loader=DataLoader(dataset=test_data,
                       shuffle=False,
                       batch_size=mini_batch)

In [8]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.linear1=torch.nn.Linear(784,512)
        self.linear2=torch.nn.Linear(512,256)
        self.linear3=torch.nn.Linear(256,128)
        self.linear4=torch.nn.Linear(128,64)
        self.linear5=torch.nn.Linear(64,10)
    def forward(self,x):
        x=x.view(-1,784)
        x=F.relu(self.linear1(x))
        x=F.relu(self.linear2(x))
        x=F.relu(self.linear3(x))
        x=F.relu(self.linear4(x))
        x=self.linear5(x)
        return x
model=Model()

In [9]:
criterion=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model.parameters(),lr=0.01,momentum=0.5)

In [10]:
if(use_gpu):
    model = model.cuda()
    criterion = criterion.cuda()

In [11]:
def train(epoch):
    running_loss=0.0
    for batch_idx,data in enumerate(train_loader):
        x,y=data
        if (use_gpu):
            x,y = x.cuda(),y.cuda()
        optimizer.zero_grad()
        y_=model(x)
        loss=criterion(y_,y)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        if batch_idx%300==0:
            print('[%d, %5d] loss: %.3f'%(epoch+1,batch_idx+1,running_loss/300))
            running_loss=0.0
def test():
    correct=0
    total=0
    with torch.no_grad():
        for data in test_loader:
            x,y=data
            if (use_gpu):
                x,y = x.cuda(),y.cuda()
            y_=model(x)
            _,predicted=torch.max(y_.data,dim=1)
            total+=y.size(0)
            correct+=(predicted==y).sum().item()
    print('Accuracy=',100*correct/total)

In [12]:
for epoch in range(10):
    train(epoch)
    test()

[1,     1] loss: 0.008
[1,   301] loss: 2.224
[1,   601] loss: 0.896
[1,   901] loss: 0.414
Accuracy= 89.44
[2,     1] loss: 0.002
[2,   301] loss: 0.310
[2,   601] loss: 0.270
[2,   901] loss: 0.233
Accuracy= 93.78
[3,     1] loss: 0.001
[3,   301] loss: 0.188
[3,   601] loss: 0.172
[3,   901] loss: 0.158
Accuracy= 95.05
[4,     1] loss: 0.001
[4,   301] loss: 0.131
[4,   601] loss: 0.120
[4,   901] loss: 0.120
Accuracy= 96.72
[5,     1] loss: 0.000
[5,   301] loss: 0.099
[5,   601] loss: 0.095
[5,   901] loss: 0.088
Accuracy= 97.04
[6,     1] loss: 0.000
[6,   301] loss: 0.073
[6,   601] loss: 0.076
[6,   901] loss: 0.076
Accuracy= 97.13
[7,     1] loss: 0.000
[7,   301] loss: 0.059
[7,   601] loss: 0.060
[7,   901] loss: 0.061
Accuracy= 97.16
[8,     1] loss: 0.001
[8,   301] loss: 0.047
[8,   601] loss: 0.051
[8,   901] loss: 0.052
Accuracy= 97.61
[9,     1] loss: 0.000
[9,   301] loss: 0.040
[9,   601] loss: 0.039
[9,   901] loss: 0.042
Accuracy= 97.54
[10,     1] loss: 0.000
[10,